## <8-2 문장 생성 모델 파인튜닝하기>   

이번 절에서는 문장 생성 모델의 데이터 전처리와 파인튜닝 과정을 실습해 보겠습니다.

### 문장 생성 모델 만들기  

<**1단계**> **코랩 노트북 초기화하기**   
이번 실습에서는 이전 실습과 마찬가지로 코랩에서 `[내 드라이브에 복사]`를 진행합니다. 그리고 코랩의 메뉴에서 [런타임 $	\rightarrow$ `런타임 유형 변경`]을 클릭하고 `[GPU]`나 `[TPU]` 둘 중 하나를 선택(GPU 권장)해 하드웨어 가속을 사용합니다.

<**2단계**> **각종 설정하기**   
다음 코드를 차례로 실행해 TPU 관련 라이브러리와 그 외에 의존성 있는 패키지를 설치하고 자신의 구글 드라이브를 코랩 노트북과 연결합니다. 다만 1단계에서 GPU를 선택했다면 코드 8-1은 생략합니다.

In [1]:
#TPU 관련 패키지 설치
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [2]:
#의존성 패키지 설치
!pip install ratsnlp

     |████████████████████████████████| 42 kB 423 kB/s 
     |████████████████████████████████| 2.8 MB 6.8 MB/s 
     |████████████████████████████████| 57 kB 4.2 MB/s 
     |████████████████████████████████| 806 kB 40.9 MB/s 
     |████████████████████████████████| 136 kB 54.4 MB/s 
     |████████████████████████████████| 398 kB 43.5 MB/s 
     |████████████████████████████████| 829 kB 56.3 MB/s 
     |████████████████████████████████| 636 kB 17.9 MB/s 
     |████████████████████████████████| 77 kB 4.7 MB/s 
     |████████████████████████████████| 895 kB 63.3 MB/s 
     |████████████████████████████████| 3.3 MB 37.5 MB/s 
     |████████████████████████████████| 1.1 MB 50.6 MB/s 
     |████████████████████████████████| 96 kB 4.5 MB/s 
     |████████████████████████████████| 144 kB 56.3 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
     |████████████████████████████████| 271 kB 59.1 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha

In [3]:
#구글 드라이브와 연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


이번 실습에서는 SK텔레콤이 공개한 KoGPT2 모델(skt/kogpt2-base-v2)을 NSMC로 파인튜닝해 볼 예정입니다. 다음 코드를 실행하면 관련 설정을 할 수 있습니다. `GenerationTrainArguments`의 인자와 역할은 <4-2>절 문서 분류 모델을 실습할 때 사용한 `GenerationTrainArguments`와 거의 비슷하므로 해당 절을 참고 바랍니다.

In [5]:
#모델 환경 설정
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments
args = GenerationTrainArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_corpus_name="nsmc",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-generation",
    max_seq_length=32,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

이어서 다음 코드를 차례로 실행해 랜덤 시드를 설정하고 각종 기록을 출력하는 로거를 설정합니다.

In [6]:
#랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [7]:
#로거 설정
nlpbook.set_logger(args)

<**3단계**> **말뭉치 내려받기**  
다음 코드를 실행하면 NSMC 말뭉치를 내려받습니다. 데이터를 내려받는 도구로 오픈소스 패키지 `Korpora`를 사용해 `corpus_name`에 해당하는 말뭉치(`nsmc`)를 `root_dir`(/root/Korpora) 아래에 저장해 둡시다.   

In [8]:
#말뭉치 다운로드
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=args.force_download,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 81.9MB/s]                           
[nsmc] download ratings_test.txt: 4.90MB [00:00, 15.7MB/s]                            


<**4단계**> **토크나이저 준비하기**   

다음 코드를 실행해 KoGPT2 모델(`skt/kogpt2-base-v2`)이 사용하는 토크나이저를 선언합니다.  
`eos_token`은 문장 마지막(end of sentence)에 붙이는 스페셜 토큰으로, SKT가 모델을 프리트레인할 때 이렇게 지정했기 때문에 같은 방식으로 사용합니다.

In [9]:
#토크나이저 준비
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
